In [40]:
import pandas as pd
import requests
import warnings

In [41]:
warnings.filterwarnings("ignore")

In [19]:
# ! pip3 install lxml html5lib

Get list of S&P 500 symbols

In [151]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
df = pd.read_html(url)[0]
symbols = df["Symbol"]
symbols

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     XYL
499     YUM
500    ZBRA
501     ZBH
502     ZTS
Name: Symbol, Length: 503, dtype: object

In [192]:
def process_frame(stock: str):
    df["Date"] = pd.to_datetime(df["Date"])
    df.set_index("Date", inplace=True)
    df = df[stock]
    if df.dtype == "O":
        mask = ~df.str.contains(" ")  # dividends contains spaces
        df = df[mask].apply(float)
    return df.to_frame()

In [205]:
def scrape_yahoo_finance(stock: str) -> pd.DataFrame:
    url = f"https://finance.yahoo.com/quote/{stock}/history/?period1=1712523837&period2=1720386234"
    r = requests.get(
        url,
        headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        },
    )
    df = pd.read_html(r.text)[0]
    df.rename(columns={"Close Close price adjusted for splits.": "Close"}, inplace=True)
    return df[["Date", "Open", "High", "Low", "Close", "Volume"]]

In [237]:
# Worked, until it stopped working
scrape_yahoo_finance("TSLA")

KeyError: "None of [Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')] are in the [columns]"

In [214]:
for idx, stock in enumerate(symbols):
    if idx % 50 == 0:
        print(f"Processing stock {stock}")
    try:
        df = scrape_yahoo_finance(stock=stock)
    except Exception as e:
        print(f"Error retrieving data for {stock} - error was {str(e)}")
        continue
    df.to_csv(f"./data/{stock}.csv", index=False)

df

Processing stock MMM
Processing stock ADP
Error retrieving data for BRK.B - error was "['Open', 'High', 'Low', 'Close', 'Volume'] not in index"
Error retrieving data for BIIB - error was "None of [Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')] are in the [columns]"
Error retrieving data for BLK - error was "None of [Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')] are in the [columns]"
Error retrieving data for BX - error was "None of [Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')] are in the [columns]"
Error retrieving data for BA - error was "None of [Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')] are in the [columns]"
Error retrieving data for BKNG - error was "None of [Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')] are in the [columns]"
Error retrieving data for BWA - error was "None of [Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], d

KeyboardInterrupt: 